# Image Generator

In [ ]:
#| default_exp image_generator

In [ ]:
#| eval:false
from Cloud2DImageConverter import data as dt, spherical_projections as sp
from matplotlib import pyplot as plt
import numpy as np
import shutil
import time
import os

Carregando os dados

In [ ]:
point_cloud = dt.load_data("../point_clouds/semantic_kitti")

Gerando as projeções
* ronny_dict: Ronny Restrepo http://ronny.rest/blog/post_2017_03_25_lidar_to_2d/* rogerio_dict: Rogério https://github.com/alunos-pfc/Spherical-Projection-SemanticKITTI


In [ ]:
ronny_dict = {
    "reflectance": [],
    "label": []
}
rogerio_dict = {
    "reflectance": [],
    "label": []
}

# Velodyne HDL 64E
h_res = 0.35
v_res = 0.4
v_fov = (-24.9, 2.0)
y_fudge = 5

for points in point_cloud:
    ronny_dict["reflectance"].append(sp.spherical_projection_1(points, h_res, v_res, v_fov, y_fudge, val="reflectance"))
    ronny_dict["label"].append(sp.spherical_projection_1(points, h_res, v_res, v_fov, y_fudge, val="label"))

    rows, cols, intensity, label = sp.spherical_projection_2(points)
    no_label, with_label = sp.create_ImageMatrix(rows, cols, intensity, label)
    rogerio_dict["reflectance"].append(no_label)
    rogerio_dict["label"].append(with_label)

In [ ]:
def mapping_colors(value_dict):
    if value_dict["val"] == "reflectance":
        return value_dict["pixel_values"]
    colors = [color_map[label] for label in value_dict["pixel_values"]]
    return colors
    
color_map = dict(dt.color_map)
for key in color_map.keys():
    color_map[key] = [round(val/255, 2) for val in color_map[key]]

In [ ]:
def ronny_image_generator(ronny_dict, output_folder, delete=True):
    start_time = time.time() 
    ronny_folder = output_folder+"ronny_images"

    # Se a pasta já existir, apague-a 
    if os.path.exists(ronny_folder) and delete:
        shutil.rmtree(ronny_folder)
    
    if not os.path.exists(ronny_folder):
        os.makedirs(ronny_folder)
        os.makedirs(ronny_folder+"/reflectance")
        os.makedirs(ronny_folder+"/label")

    cmap = "jet"
    dpi = 100
    # Jutando os as listas de reflectância e label, agora zip_dict[0] = ronny_dict["reflectance"]
    for zip_dict in zip(ronny_dict["reflectance"], ronny_dict["label"]):
        # Iterando entre ronny_dict["reflectance"] e ronny_dict["label"]
        for value_dict in zip_dict:
            fig, ax = plt.subplots(figsize=(value_dict["x_max"]/dpi, value_dict["y_max"]/dpi), dpi=dpi)
            ax.scatter(value_dict["x_img"],value_dict["y_img"], s=1, c=mapping_colors(value_dict), linewidths=0, alpha=1)
            ax.set_facecolor((0, 0, 0)) 
            ax.axis('scaled')              
            ax.xaxis.set_visible(False)    
            ax.yaxis.set_visible(False)    
            plt.xlim([0, value_dict["x_max"]])   
            plt.ylim([0, value_dict["y_max"]])  

            # value_dict["val"] é o valor de reflectance ou label
            folder_type = "/" + value_dict["val"]
            
            # Caminho final para salver
            final_path = os.path.join(ronny_folder+folder_type)
            
            # Define o nome do arquivo, tais como 000001, 000002...
            file_number = len(os.listdir(final_path)) + 1
            file_name = f"{final_path}/{file_number:06d}.png"
            
            plt.savefig(file_name, bbox_inches='tight', pad_inches=0, format='png')
            plt.close()

    end_time = time.time()  # Registra o tempo de fim da execução
    elapsed_time = end_time - start_time
    print(f"Tempo de execução: {elapsed_time} segundos")

ronny_image_generator(ronny_dict, output_folder="../point_clouds/")

In [ ]:
def rogerio_image_generator(rogerio_dict, output_folder, delete=True):
    start_time = time.time() 
    rogerio_folder =  output_folder+"rogerio_images"

    # Se a pasta já existir, apague-a 
    if os.path.exists(rogerio_folder) and delete:
        shutil.rmtree(rogerio_folder)
        
    os.makedirs(rogerio_folder)
    os.makedirs(rogerio_folder+"/reflectance")
    os.makedirs(rogerio_folder+"/label")

    for zip_dict in zip(rogerio_dict["reflectance"], rogerio_dict["label"]):
        for i, value in enumerate(rogerio_dict.keys()): 
            if value == "reflectance":
                cmap = "jet"
            else:
                cmap = None
                
            fig, ax = plt.subplots(figsize=(10,5))
            # Plote a primeira matriz no primeiro subplot
            ax.imshow(zip_dict[i], cmap=cmap)
            ax.xaxis.set_visible(False)    
            ax.yaxis.set_visible(False)  

            # dict["val"] é o valor de reflectance ou label
            folder_type = "/" + value

            # Caminho final para salver
            final_path = os.path.join(rogerio_folder+folder_type)

            # Define o nome do arquivo, tais como 000001, 000002...
            file_number = len(os.listdir(final_path)) + 1
            file_name = f"{final_path}/{file_number:06d}.png"

            plt.savefig(file_name, bbox_inches='tight', pad_inches=0, format='png')
            plt.close()
            
    end_time = time.time()  # Registra o tempo de fim da execução
    elapsed_time = end_time - start_time
    print(f"Tempo de execução: {elapsed_time} segundos")

rogerio_image_generator(rogerio_dict, output_folder="../point_clouds/")